### Prerequisites
Install HuggingFace packages and create submission directory.

In [1]:
!rm -rf /kaggle/working/submission # WARNING: 삭제가 필요한 경우에만 사용

!mkdir /kaggle/working/submission

In [2]:
%%time
import os, sys, subprocess

sys.path.insert(0, "/kaggle/working/submission/lib") # 설치 경로 추가

def uninstall_package(package_name):
    """패키지를 삭제하는 함수"""
    os.system(f"pip uninstall -y {package_name}")

os.system("pip install -q -U pip")

packages = ["bitsandbytes", "accelerate", "transformers"] # 설치할 패키지 목록
for pkg in packages:
    try:
        print(f"{pkg}을(를) 설치합니다.")
        subprocess.check_call([sys.executable, "-m", "pip", "install", pkg])
    except subprocess.CalledProcessError:
        print(f"## {pkg} 설치에 실패하였으므로 프로세스를 종료합니다.")
        sys.exit(1)  # 비정상 종료
     
os.system("pip freeze | egrep 'bitsandbytes|accelerate|transformers'") # 설치된 특정 패키지 목록을 출력합니다.

os.system("pip cache purge") # pip 캐시를 정리합니다.

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 24.6.1 requires cubinlinker, which is not installed.
cudf 24.6.1 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 24.6.1 requires ptxcompiler, which is not installed.
cuml 24.6.1 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 24.6.1 requires cupy-cuda11x>=12.0.0, which is not installed.
keras-cv 0.9.0 requires keras-core, which is not installed.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
ucx-py 0.38.0 requires libucx<1.16,>=1.15.0, which is not installed.
ucxx 0.38.0 requires libucx>=1.15.0, which is not installed.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.8 which is incompatible.
apache-beam 2.46.0 requires numpy<1.25.0,>=1.14.3, but you have numpy 1.26.4 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10

Files removed: 242
CPU times: user 11.7 ms, sys: 4.73 ms, total: 16.5 ms
Wall time: 2min 53s


### HuggingFace Login

Add HuggingFace access token to secrets. You can find it in `Add-ons -> secrets`

In [3]:
import huggingface_hub
from kaggle_secrets import UserSecretsClient

huggingface_hub.login(token=UserSecretsClient().get_secret("HF_TOKEN"))

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


### Download Model via HuggingFace
In this notebook, we are using gemma-2-9b model with 4-bit quantization.

In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch

torch.backends.cuda.enable_mem_efficient_sdp(False) # 메모리 효율적인 SDP(Sparse Dense Packing) 비활성화
torch.backends.cuda.enable_flash_sdp(False) # Flash SDP 비활성화

model_id = "google/gemma-2-9b-it" # 모델 식별자 설정

bnb_config = BitsAndBytesConfig(
    load_in_4bit = True,  # 4비트 로딩 활성화
    bnb_4bit_compute_dtype=torch.float16,  # 계산에 사용할 데이터 타입을 float16으로 설정
)

model = AutoModelForCausalLM.from_pretrained( # 사전 훈련된 모델 로드
    model_id,
    quantization_config = bnb_config,  # 양자화 설정 적용
    torch_dtype = torch.float16,  # 모델의 데이터 타입을 float16으로 설정
    device_map = "auto",  # 장치 맵 자동 설정
    trust_remote_code = True,  # 원격 코드 신뢰 설정
)

tokenizer = AutoTokenizer.from_pretrained(model_id) # 토크나이저 로드


Unused kwargs: ['bnb_4bit_quanty_type', 'bnb_4bit_use_double_quanty']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


config.json:   0%|          | 0.00/857 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/39.1k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.90G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/40.6k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

### Save Model
Save the loaded model and tokenizer in the submission directory.
Remove the model and tokenizer from the memory.

In [5]:
model.save_pretrained("/kaggle/working/submission/model") # 모델을 지정된 경로에 저장합니다.
tokenizer.save_pretrained("/kaggle/working/submission/model") # 토크나이저를 같은 경로에 저장합니다.

('/kaggle/working/submission/model/tokenizer_config.json',
 '/kaggle/working/submission/model/special_tokens_map.json',
 '/kaggle/working/submission/model/tokenizer.model',
 '/kaggle/working/submission/model/added_tokens.json',
 '/kaggle/working/submission/model/tokenizer.json')

In [6]:
import gc, torch
del model, tokenizer # 사용이 끝난 model과 tokenizer 객체를 메모리에서 삭제합니다.
gc.collect() # 가비지 컬렉터를 수동으로 호출하여, 참조되지 않는 객체들을 메모리에서 제거합니다.
torch.cuda.empty_cache() # PyTorch의 CUDA 캐시를 비워 GPU 메모리를 확보합니다.

## Agent

In [7]:
%%writefile /kaggle/working/submission/main.py
# Setup
import os
import sys

# **중요:** 시스템 경로를 아래와 같이 설정하여 코드가 노트북과 시뮬레이션 환경 양쪽에서 모두 작동하도록 합니다.

KAGGLE_AGENT_PATH = "/kaggle_simulations/agent/" # Kaggle 에이전트가 위치한 경로 정의

# Kaggle 에이전트 경로가 존재하는지 확인하고, 'lib' 디렉토리를 시스템 경로에 추가합니다.
# 이는 코드가 Kaggle 노트북과 Kaggle 시뮬레이션 환경 양쪽에서 모두 작동할 수 있도록 보장합니다.
if os.path.exists(KAGGLE_AGENT_PATH):
    sys.path.insert(0, os.path.join(KAGGLE_AGENT_PATH, 'lib'))
else:
    sys.path.insert(0, "/kaggle/working/submission/lib")

import contextlib
import os
import sys
from pathlib import Path

from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch

torch.backends.cuda.enable_mem_efficient_sdp(False) # 메모리 효율적인 SDP를 비활성화합니다.
torch.backends.cuda.enable_flash_sdp(False) # Flash SDP를 비활성화합니다.

# KAGGLE_AGENT_PATH가 존재하는지 확인하고, 존재하면 모델 경로를 설정합니다.
if os.path.exists(KAGGLE_AGENT_PATH):
    MODEL_PATH = os.path.join(KAGGLE_AGENT_PATH, "model")
else:
    MODEL_PATH = "/kaggle/working/submission/model"

# 프롬프트 포매팅을 위한 코드
import itertools
from typing import Iterable

class GemmaFormatter:
    _start_token = '<start_of_turn>'
    _end_token = '<end_of_turn>'

    def __init__(self, system_prompt: str = None, few_shot_examples: Iterable = None):
        # 생성자에서 시스템 프롬프트와 예시를 초기화합니다.
        self._system_prompt = system_prompt
        self._few_shot_examples = few_shot_examples
        self._turn_user = f"{self._start_token}user\n{{}}{self._end_token}\n"
        self._turn_model = f"{self._start_token}model\n{{}}{self._end_token}\n"
        self.reset()

    def __repr__(self):
        # 객체를 문자열로 표현할 때 상태를 반환합니다.
        return self._state

    def user(self, prompt):
        # 사용자 프롬프트를 상태에 추가합니다.
        self._state += self._turn_user.format(prompt)
        return self

    def model(self, prompt):
        # 모델 프롬프트를 상태에 추가합니다.
        self._state += self._turn_model.format(prompt)
        return self

    def start_user_turn(self):
        # 사용자 턴 시작을 상태에 추가합니다.
        self._state += f"{self._start_token}user\n"
        return self

    def start_model_turn(self):
        # 모델 턴 시작을 상태에 추가합니다.
        self._state += f"{self._start_token}model\n"
        return self

    def end_turn(self):
        # 턴 종료를 상태에 추가합니다.
        self._state += f"{self._end_token}\n"
        return self

    def reset(self):
        # 상태를 초기화하고, 시스템 프롬프트와 예시가 있다면 적용합니다.
        self._state = ""
        if self._system_prompt is not None:
            self.user(self._system_prompt)
        if self._few_shot_examples is not None:
            self.apply_turns(self._few_shot_examples, start_agent='user')
        return self

    def apply_turns(self, turns: Iterable, start_agent: str):
        # 주어진 턴들을 순서대로 적용합니다. 시작 에이전트에 따라 순서가 결정됩니다.
        formatters = [self.model, self.user] if start_agent == 'model' else [self.user, self.model]
        formatters = itertools.cycle(formatters)
        for fmt, turn in zip(formatters, turns):
            fmt(turn)
        return self


# Agent 정의
import re


@contextlib.contextmanager # 텐서 타입을 임시로 설정하는 컨텍스트 매니저
def _set_default_tensor_type(dtype: torch.dtype):
    """주어진 dtype으로 기본 torch dtype을 설정합니다."""
    torch.set_default_dtype(dtype)
    yield
    torch.set_default_dtype(torch.float)

class GemmaAgent:
    def __init__(self, model_path=MODEL_PATH, device='cuda:0', system_prompt=None, few_shot_examples=None):
        self._device = torch.device(device) # 디바이스 설정
        self.formatter = GemmaFormatter(system_prompt=system_prompt, few_shot_examples=few_shot_examples) # 포맷터 초기화

        print("Initializing model")
        self.model = AutoModelForCausalLM.from_pretrained(
            model_path,
            device_map = "auto",
        ) # 모델 로드
        
        self.tokenizer = AutoTokenizer.from_pretrained(model_path) # 토크나이저 로드

    def __call__(self, obs, *args):
        self._start_session(obs) # 세션 시작
        prompt = str(self.formatter) # 프롬프트 생성
        response = self._call_llm(prompt) # LLM 호출
        response = self._parse_response(response, obs) # 응답 파싱
        print(f"{response=}")
        return response

    def _start_session(self, obs: dict):
        # 세션 시작 메서드 (구현 필요)
        raise NotImplementedError

    def _call_llm(self, prompt, max_new_tokens=32, **sampler_kwargs):
        # LLM 호출 메서드
        if sampler_kwargs is None:
            sampler_kwargs = {
                'temperature': 0.01,
                'top_p': 0.1,
                'top_k': 1,
        }
        input_ids = self.tokenizer(prompt, return_tensors="pt").to("cuda")
        outputs = self.model.generate(**input_ids, max_new_tokens=max_new_tokens, kwargs=sampler_kwargs)
        return self.tokenizer.decode(outputs[0])

    def _parse_keyword(self, response: str):
        # 응답에서 키워드 파싱
        match = re.search(r"(?<=\*\*)([^*]+)(?=\*\*)", response)
        if match is None:
            keyword = ''
        else:
            keyword = match.group().lower()
        return keyword

    def _parse_response(self, response: str, obs: dict):
        # 응답 파싱 메서드 (구현 필요)
        raise NotImplementedError

def interleave_unequal(x, y):
    """두 리스트의 요소를 교차로 결합합니다. 리스트 길이가 다를 경우 None을 제외하고 결합합니다."""
    return [
        item for pair in itertools.zip_longest(x, y) for item in pair if item is not None
    ]


class GemmaQuestionerAgent(GemmaAgent):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        # GemmaAgent 클래스를 상속받아 초기화합니다. 추가 인자들은 부모 클래스의 초기화 메서드로 전달됩니다.

    def _start_session(self, obs):
        self.formatter.reset() # 대화 세션을 시작하기 전에 formatter를 초기화합니다.
        self.formatter.user("Let's play 20 Questions. You are playing the role of the Questioner.") # 사용자에게 20 Questions 게임을 시작한다고 알리고, 사용자가 질문자 역할임을 알립니다.
        turns = interleave_unequal(obs.questions, obs.answers) # 질문과 답변을 교차로 배치합니다.
        self.formatter.apply_turns(turns, start_agent='model') # 교차 배치된 질문과 답변을 formatter에 적용합니다. 모델이 첫 번째로 시작합니다.
        
        if obs.turnType == 'ask':
            self.formatter.user("Please ask a yes-or-no question.") # 사용자에게 예/아니오 질문을 요청합니다.
        elif obs.turnType == 'guess':
            self.formatter.user("Now guess the keyword. Surround your guess with double asterisks.") # 사용자에게 키워드를 추측하도록 요청합니다. 추측은 이중 별표로 둘러싸야 합니다.
        self.formatter.start_model_turn() # 모델의 차례를 시작합니다.

    def _parse_response(self, response: str, obs: dict):
        if obs.turnType == 'ask':
            match = re.search(".+?\?", response.replace('*', '')) # 사용자의 응답에서 질문을 찾습니다. 별표는 제거됩니다.
            if match is None:
                question = "Is it a place?" # 질문을 찾지 못한 경우 기본 질문을 사용합니다.
            else:
                question = match.group() # 찾은 질문을 반환합니다.
            return question
        elif obs.turnType == 'guess':
            guess = self._parse_keyword(response) # 사용자의 응답에서 키워드를 추출합니다.
            return guess
        else:
            raise ValueError("Unknown turn type:", obs.turnType) # 알 수 없는 차례 유형에 대한 예외를 발생시킵니다.

            
class GemmaAnswererAgent(GemmaAgent):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs) # GemmaAgent 클래스를 상속받아 초기화합니다. 추가 인자들은 부모 클래스의 초기화 메서드로 전달됩니다.

    def _start_session(self, obs):
        self.formatter.reset() # 대화 세션을 시작하기 전에 formatter를 초기화합니다.
        
        # 사용자에게 20 Questions 게임을 시작한다고 알리고, 사용자가 답변자 역할임을 알립니다. 키워드와 카테고리도 알립니다.
        self.formatter.user(f"Let's play 20 Questions. You are playing the role of the Answerer. The keyword is {obs.keyword} in the category {obs.category}.")
        
        turns = interleave_unequal(obs.questions, obs.answers) # 질문과 답변을 교차로 배치합니다.
        self.formatter.apply_turns(turns, start_agent='user') # 교차 배치된 질문과 답변을 formatter에 적용합니다. 사용자가 첫 번째로 시작합니다.
        
        # 사용자에게 키워드와 카테고리에 관한 질문에 예/아니오로 답변하도록 요청합니다. 답변은 이중 별표로 둘러싸야 합니다.
        self.formatter.user(f"The question is about the keyword {obs.keyword} in the category {obs.category}. Give yes-or-no answer and surround your answer with double asterisks, like **yes** or **no**.")
        self.formatter.start_model_turn() # 모델의 차례를 시작합니다.

    def _parse_response(self, response: str, obs: dict):
        answer = self._parse_keyword(response) # 사용자의 응답에서 키워드를 추출합니다.
        return 'yes' if 'yes' in answer else 'no'

# Agent Creation
system_prompt = "You are an AI assistant designed to play the 20 Questions game. In this game, the Answerer thinks of a keyword and responds to yes-or-no questions by the Questioner. The keyword is a specific person, place, or thing."

few_shot_examples = [
    "Let's play 20 Questions. You are playing the role of the Questioner. Please ask your first question.",
    "Is it a thing?", "**no**",
    "Is is a place?", "**yes**",
    "Is it a country?", "**yes**",
    "Does it start with f?", "**yes** Now guess the keyword.",
    "**France**", "Correct!",
]


# **중요:** 에이전트를 전역 변수로 정의하여 필요한 에이전트만 로드합니다.
# 두 에이전트를 모두 로드하면 OOM(Out of Memory)이 발생할 가능성이 높습니다.
agent = None


def get_agent(name: str):
    global agent
    
    # 에이전트가 None이고, 요청된 이름이 'questioner'인 경우
    if agent is None and name == 'questioner':
        # 질문자 에이전트를 초기화합니다.
        agent = GemmaQuestionerAgent(
            device='cuda:0',
            system_prompt=system_prompt,
            few_shot_examples=few_shot_examples,
        )
    # 에이전트가 None이고, 요청된 이름이 'answerer'인 경우
    elif agent is None and name == 'answerer':
        # 답변자 에이전트를 초기화합니다.
        agent = GemmaAnswererAgent(
            device='cuda:0',
            system_prompt=system_prompt,
            few_shot_examples=few_shot_examples,
        )
    # 에이전트가 초기화되지 않았다면 에러를 발생시킵니다.
    assert agent is not None, "Agent not initialized."

    return agent


def agent_fn(obs, cfg):
    # 관찰된 턴 타입에 따라 적절한 에이전트를 호출합니다.
    if obs.turnType == "ask":
        response = get_agent('questioner')(obs)
    elif obs.turnType == "guess":
        response = get_agent('questioner')(obs)
    elif obs.turnType == "answer":
        response = get_agent('answerer')(obs)
        
    # 응답이 None이거나 길이가 1 이하인 경우 'yes'를 반환합니다.
    if response is None or len(response) <= 1:
        return "yes"
    else:
        return response

Writing /kaggle/working/submission/main.py


In [8]:
!apt install pigz pv > /dev/null

In [9]:
!tar --use-compress-program='pigz --fast --recursive | pv' -cf /kaggle/working/submission.tar.gz -C /kaggle/working/submission .

8.11GiB 0:03:16 [42.2MiB/s] [  <=>                                             ]


## Simulate Game

### Load test data
Download the latest keywords.py from [kaggle-environments](https://github.com/Kaggle/kaggle-environments/blob/master/kaggle_environments/envs/llm_20_questions/keywords.py) github repo

In [10]:
%%bash

wget -O keywords.py https://raw.githubusercontent.com/Kaggle/kaggle-environments/master/kaggle_environments/envs/llm_20_questions/keywords.py
mkdir -p /kaggle/working/simulation/

--2024-07-21 04:25:05--  https://raw.githubusercontent.com/Kaggle/kaggle-environments/master/kaggle_environments/envs/llm_20_questions/keywords.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 93552 (91K) [text/plain]
Saving to: 'keywords.py'

     0K .......... .......... .......... .......... .......... 54% 3.00M 0s
    50K .......... .......... .......... .......... .         100% 27.6M=0.02s

2024-07-21 04:25:05 (5.03 MB/s) - 'keywords.py' saved [93552/93552]



In [11]:
import json
import pandas as pd
import numpy as np
from keywords import KEYWORDS_JSON

def create_keyword_df(KEYWORDS_JSON):
    json_data = json.loads(KEYWORDS_JSON) # JSON 문자열을 파이썬 객체로 변환

    # 키워드, 카테고리, 대체어를 저장할 리스트 초기화
    keyword_list = []
    category_list = []
    alts_list = []

    # JSON 데이터를 순회하며 키워드, 카테고리, 대체어 정보 추출
    for i in range(len(json_data)):
        for j in range(len(json_data[i]['words'])):
            keyword = json_data[i]['words'][j]['keyword']
            keyword_list.append(keyword)
            category_list.append(json_data[i]['category'])
            alts_list.append(json_data[i]['words'][j]['alts'])

    # 추출한 정보를 이용해 pandas DataFrame 생성
    data_pd = pd.DataFrame(columns=['keyword', 'category', 'alts'])
    data_pd['keyword'] = keyword_list
    data_pd['category'] = category_list
    data_pd['alts'] = alts_list
    
    return data_pd


In [12]:
keywords = create_keyword_df(KEYWORDS_JSON) # KEYWORDS_JSON에서 키워드 데이터를 DataFrame으로 생성합니다.

# keywords_df.head(5)
keywords.tail(5)

,keyword,category,alts
1137,rhine,place,[]
1138,yangtze river,place,"[changjiang, yangtze]"
1139,yellow river,place,[huang he]
1140,zambezi river,place,[zambezi]
1141,yenisei river,place,[yenisei]


In [13]:
keywords["category"].unique() # keywords DataFrame의 "category" 열에서 고유한 값을 추출합니다.

array(['things', 'place'], dtype=object)

In [14]:
# 키워드를 CSV 파일로 저장합니다. 인덱스는 포함하지 않습니다.
keywords.to_csv('/kaggle/working/simulation/keywords.csv', index=False)

### Create Agents
2 vs 2

In [15]:
%%writefile /kaggle/working/simulation/agent1.py

import pandas as pd
import numpy as np

# 키워드 파일에서 키워드를 불러옵니다.
keywords = pd.read_csv("/kaggle/working/simulation/keywords.csv").keyword.values

def agent_fn(obs, cfg):
    global keywords
    
    # 현재 라운드 번호를 표시합니다.
    k = len( obs.questions )
    if obs.turnType == "ask":
        print()
        print("#"*25)
        print(f"### Round {k+1}")
        print("#"*25)

    # 에이전트 이름과 JSON 입력을 표시합니다.
    name = "Team 1 - Questioner - Agent Random"
    print(f"\n{name}\nINPUT =",obs)
    
    # 응답을 생성합니다.
    keyword = np.random.choice(keywords)
    if obs.turnType == "ask": # 질문 차례인 경우
        response = f"Is it {keyword}?" # "Is it {선택된 키워드}?" 형태의 질문 생성
    else: #obs.turnType == "guess"
        response = keyword # 선택된 키워드를 응답으로 설정
        if obs.answers[-1] == "yes": # 마지막 답변이 'yes'인 경우
            response = obs.questions[-1].rsplit(" ",1)[1][:-1] # 마지막 질문에서 키워드를 추출하여 응답으로 설정
    print(f"OUTPUT = '{response}'") # 생성된 응답 출력

    return response

Writing /kaggle/working/simulation/agent1.py


In [16]:
%%writefile /kaggle/working/simulation/agent2.py

import numpy as np

def agent_fn(obs, cfg):
    
    # DISPLAY AGENT NAME AND JSON INPUT
    name = "Team 1 - Answerer - Agent Random"
    print(f"\n{name}\nINPUT =",obs)
    
    # GENERATE RESPONSE
    response = "no"
    #response = np.random.choice(["yes","no"])
    # 질문에 키워드가 포함되어 있으면 응답을 "yes"로 변경합니다.
    if obs.keyword.lower() in obs.questions[-1].lower():
        response = "yes"
    print(f"OUTPUT = '{response}'")

    return response

Writing /kaggle/working/simulation/agent2.py


In [17]:
%%writefile /kaggle/working/simulation/agent3.py

import pandas as pd
import numpy as np

keywords = pd.read_csv("/kaggle/working/simulation/keywords.csv").keyword.values

def agent_fn(obs, cfg):
    global keywords
    
    # DISPLAY AGENT NAME AND JSON INPUT
    name = "Team 2 - Questioner - Agent Random"
    print(f"\n{name}\nINPUT =",obs)
    
    # GENERATE RESPONSE
    # 응답 생성
    keyword = np.random.choice(keywords)  # 키워드 중 하나를 무작위로 선택
    if obs.turnType == "ask":  # 질문 차례인 경우
        response = f"Is it {keyword}?"  # "Is it {선택된 키워드}?" 형태의 질문 생성
    else:  # 추측 차례인 경우
        response = keyword  # 선택된 키워드를 응답으로 설정
        if obs.answers[-1] == "yes":  # 마지막 답변이 'yes'인 경우
            response = obs.questions[-1].rsplit(" ",1)[1][:-1]  # 마지막 질문에서 키워드를 추출하여 응답으로 설정
    print(f"OUTPUT = '{response}'")  # 생성된 응답 출력

    return response

Writing /kaggle/working/simulation/agent3.py


In [18]:
%%writefile /kaggle/working/simulation/agent4.py

import numpy as np

def agent_fn(obs, cfg):
    
    # DISPLAY AGENT NAME AND JSON INPUT
    name = "Team 2 - Answerer - Agent Random"
    print(f"\n{name}\nINPUT =",obs)
    
    # GENERATE RESPONSE
    response = "no"
    #response = np.random.choice(["yes","no"])
    # 질문에 키워드가 포함되어 있으면 응답을 "yes"로 변경합니다.
    if obs.keyword.lower() in obs.questions[-1].lower():
        response = "yes"
    print(f"OUTPUT = '{response}'")

    return response

Writing /kaggle/working/simulation/agent4.py


### Create Environment

In [19]:
!pip install -q pygame

In [ ]:
GEMMA_AS_QUESTIONER = True # GEMMA가 질문자로 활동하는지 여부
GEMMA_AS_ANSWERER = True # GEMMA가 답변자로 활동하는지 여부

from kaggle_environments import make
env = make("llm_20_questions", debug=True) # 'llm_20_questions' 환경을 생성하고 디버그 모드 활성화

# TEAM 1
agent1 = "/kaggle/working/simulation/agent1.py" # 팀 1의 첫 번째 에이전트 경로
agent2 = "/kaggle/working/simulation/agent2.py" # 팀 1의 두 번째 에이전트 경로

# TEAM 2 - QUESTIONER
agent3 = "/kaggle/working/simulation/agent3.py" # 팀 2의 질문자 에이전트 기본 경로
if GEMMA_AS_QUESTIONER: # GEMMA가 질문자로 활동하는 경우
    agent3 = "/kaggle/working/submission/main.py" # GEMMA의 질문자 에이전트 경로로 변경
    
# TEAM 2 - ANSWERER
agent4 = "/kaggle/working/simulation/agent4.py" # 팀 2의 답변자 에이전트 기본 경로
if GEMMA_AS_ANSWERER: # GEMMA가 답변자로 활동하는 경우
    agent4 = "/kaggle/working/submission/main.py" # GEMMA의 답변자 에이전트 경로로 변경
    
env.reset()
log = env.run([agent1, agent2, agent3, agent4]) # 에이전트들을 실행하고 로그를 기록

env.render(mode="ipython", width=600, height=500) # 결과를 IPython 모드로 렌더링, 크기 지정

import gc, torch
del make, env, log # 사용한 리소스 삭제
gc.collect() # 가비지 컬렉션 실행
torch.cuda.empty_cache() # PyTorch CUDA 캐시를 비움

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.



#########################
### Round 1
#########################

Team 1 - Questioner - Agent Random
INPUT = {'remainingOverageTime': 300, 'step': 0, 'questions': [], 'guesses': [], 'answers': [], 'role': 'guesser', 'turnType': 'ask', 'keyword': '', 'category': ''}
OUTPUT = 'Is it Duct tape?'


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

2024-07-21 04:26:35.933292: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-21 04:26:35.933390: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-21 04:26:36.075130: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


Initializing model
response='Is it a thing?'

Team 1 - Answerer - Agent Random
INPUT = {'remainingOverageTime': 300, 'questions': ['Is it Duct tape?'], 'guesses': [], 'answers': [], 'role': 'answerer', 'turnType': 'answer', 'keyword': 'miami florida', 'category': 'place', 'step': 1}
OUTPUT = 'no'


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Initializing model
response='no'

Team 1 - Questioner - Agent Random
INPUT = {'remainingOverageTime': 300, 'step': 2, 'questions': ['Is it Duct tape?'], 'guesses': [], 'answers': ['no'], 'role': 'guesser', 'turnType': 'guess', 'keyword': '', 'category': ''}
OUTPUT = 'Ointment'
response='no'

#########################
### Round 2
#########################

Team 1 - Questioner - Agent Random
INPUT = {'remainingOverageTime': 300, 'step': 3, 'questions': ['Is it Duct tape?'], 'guesses': ['Ointment'], 'answers': ['no'], 'role': 'guesser', 'turnType': 'ask', 'keyword': '', 'category': ''}
OUTPUT = 'Is it wellington new zealand?'
